## Laboratorio 04 - SQL parte 3

### Davis Alvarez  - 15842

#### Bienvenida

Bienvenido a su tercer laboratorio de Bases de datos.

El objetivo de este laboratorio es que usted profundice y practique sobre los conceptos de queries anidados (_subqueries_) y la aplicación del concepto de agregación y de las funciones de agregación. Usted deberá conocer la sintaxis de un `SELECT` introduciendo los conceptos de agregación (`GROUP BY`) y filtros sobre datos agregados (`HAVING`).

Para este laboratorio utilizaremos la base de datos **flights-lab3.db** utilizada durante el laboratorio 3.

Recuerde que puede encontrar más información acerca de los campos en: https://www.transtats.bts.gov/DL_SelectFields.asp?Table_ID=236

In [2]:
%load_ext sql

In [3]:
sql sqlite:///flights-lab3.db

'Connected: @flights-lab3.db'

### Ejercicio 1: Consultas SQL (100 puntos)

Prepare un query que responda a cada una de las siguientes preguntas:

In [5]:
%sql PRAGMA table_info(ontime);

 * sqlite:///flights-lab3.db
Done.


cid,name,type,notnull,dflt_value,pk
0,Year,int,0,None,0
1,Month,int,0,None,0
2,DayofMonth,int,0,None,0
3,DayOfWeek,int,0,None,0
4,DepTime,int,0,None,0
5,CRSDepTime,int,0,None,0
6,ArrTime,int,0,None,0
7,CRSArrTime,int,0,None,0
8,UniqueCarrier,varchar(5),0,None,0
9,FlightNum,int,0,None,0


In [6]:
%sql PRAGMA table_info(airports);

 * sqlite:///flights-lab3.db
Done.


cid,name,type,notnull,dflt_value,pk
0,iata,varchar(4),0,None,0
1,airport,varchar(100),0,None,0
2,city,varchar(100),0,None,0
3,state,varchar(4),0,None,0
4,country,varchar(4),0,None,0
5,lat,double,0,None,0
6,long,double,0,None,0


#### Pregunta 1: ¿Qué aerolineas vuelan de California a Nueva York?

Su respuesta debe mostrar el nombre de la aerolinea.

De su respuesta utilizando subqueries.

 * sqlite:///flights-lab3.db
Done.


UniqueCarrier
B6
UA
DL
AA


_Hint:_ Deben ser cuatro aerolineas

#### Pregunta 2: ¿Qué aerolineas vuelan de California a Nueva York y también de California a Nuevo México?

Su respuesta debe mostrar el nombre de las aerolineas

De su respuesta en términos de subqueries.

In [33]:
%%sql SELECT DISTINCT UniqueCarrier
        FROM (SELECT DISTINCT UniqueCarrier, O.origin, O.dest, A.state
            FROM ontime O left join airports A on A.iata = O.origin
            WHERE A.state='CA') AS cali
            left join airports llegada on llegada.iata = cali.dest
            WHERE llegada.state='NM' OR llegada.state='NY'

 * sqlite:///flights-lab3.db
Done.


UniqueCarrier
B6
WN
OO
UA
DL
AA
XE


#### Pregunta 3. ¡Quiero maximizar mi millas! (15 puntos)

Suponga que en un futuro no muy lejano le gustaría visitar tanto Portland (PDX) como Eugene (EUG), saliendo desde SFO. Con el objetivo de maximizar mis millas de viajero frecuente, me gustaría utilizar la misma aerolínea para ambos vuelos.

Escriba un query que muestre los nombres de las aerolíneas que realizaron tanto vuelos SFO -> PDX como vuelos SFO -> EUG.
Asegúrese de mostrar el nombre de las aerolíneas sin duplicados.

In [10]:
%%sql SELECT DISTINCT UniqueCarrier, sfo.origin, sfo.dest
        FROM (SELECT O.UniqueCarrier, O.origin, O.dest, A.state
            FROM ontime O left join airports A on A.iata = O.origin
            WHERE A.iata='SFO' AND A.state='CA') AS sfo
            left join airports llegada on llegada.iata = sfo.dest
            WHERE sfo.dest='EUG' OR sfo.dest='PDX' AND sfo.state='CA'

 * sqlite:///flights-lab3.db
Done.


UniqueCarrier,origin,dest
OO,SFO,EUG
UA,SFO,PDX
AS,SFO,PDX
UA,SFO,EUG
OO,SFO,PDX


_Respuesta:_
```
SkyWest Airlines Inc.
United Air Lines Inc.
```

In [46]:
%%sql SELECT DISTINCT UniqueCarrier, O.origin, O.dest, A.state
            FROM ontime O left join airports A on A.iata = O.origin
            WHERE A.iata='SFO'

 * sqlite:///flights-lab3.db
Done.


UniqueCarrier,Origin,Dest,state
WN,SFO,LAS,CA
WN,SFO,LAX,CA
WN,SFO,MDW,CA
WN,SFO,SAN,CA
XE,SFO,LAX,CA
OO,SFO,SLC,CA
OO,SFO,SMF,CA
OO,SFO,MFR,CA
OO,SFO,SBP,CA
OO,SFO,FAT,CA


In [55]:
%sql SELECT * FROM airlines where airlineCode='OO' or airlineCode='UA' or airlineCode='SA'

 * sqlite:///flights-lab3.db
Done.


AirlineCode,AirlineName
OO,SkyWest Airlines Inc.
SA,South African Airways
UA,United Air Lines Inc.


In [6]:
%sql select * from airports where iata='PDX'

 * sqlite:///flights-lab3.db
Done.


iata,airport,city,state,country,lat,long
PDX,Portland Intl,Portland,OR,USA,45.58872222,-122.5975


#### Pregunta 4: ¿Existen aerolineas que hagan únicamente vuelos internos?

Un vuelo interno es aquel cuyo aeropuerto origen y aeropuerto destino se encuentran en el mismo estado.

De su respuesta en términos de `EXCEPT`.

_Respuesta:_ En la base de datos no hay ninguna aerolinea que haga únicamente vuelos internos.

#### Pregunta 5: ¿Qué proporción de aerolineas se retrasan en llegar un promedio 10 minutos o más con respecto al total de aerolíneas que han operado vuelos? (20 puntos)

En el laboratorio 3 usted desarrolló un query para mostrar qué aerolineas se retrasan en llegar un promedio 10 minutos o más.

Escriba ahora un query que retorne la proporción de las aerolíneas que se retrasan en promedio 10 minutos o más, con respecto a la cantidad de aerolíneas que operaron vuelos.

Recuerde no quemar (_hardcode_) la cantidad de aerolíneas disponibles, sino obtener la cantidad a partir de un query.

**Nota**: la función `COUNT(*)` en SQLite retorna valores de tipo integer, por lo que deberá investigar e incluir en su query una conversión (_casteing_) de valores utilizando la función `CAST`: `SELECT CAST (COUNT(*) AS float)` o bien un cast implícito derivado de una operación: `COUNT(*) * 1.0`.


_Respuesta: La proporción de aerolineas que se retrasan más de 10 minutos en promedio representa el 40% de las aerolineas que operan vuelos_

#### Pregunta 6: ¿Cómo se relacionan los retrasos en el tiempo de salida (departure) con los retrasos en el tiempo de llegada (arrival)? (30 puntos)

Si un vuelo se retrasa al salir, nos interesa estudiar qué tanto afecta esto al tiempo de llegada.

Una forma de estudiar la variación conjunta de dos variables es utilizar la _covarianza muestral_.

(Sí, por favor refrésquelo: https://www.google.com/search?q=covarianza+muestral)

Mientras más alto sea el valor de la covarianza muestral entre dos variables, mayor será la tendencia de estas a comportarse de la misma forma. Por el contrario, una covarianza negativa indica que las variables tienden a estar inversamente relacionadas.

Una forma de computar la covarianza es como sigue:

\begin{equation}
    Cov (X, Y) = \dfrac{1}{n-1}\sum_{i=1}^{n}(x_i - \overline{x})(y_i - \overline{y})
\end{equation}

En donde $x_i$ denota la $i$ muestra de la variable $X$, $y_i$ la muestra $i$ de la variable $Y$; mientras que  las medias de $X$ y $Y$ son denotadas por $\overline{x}$ y $\overline{y}$ respectivamente.

El objetivo de este ejercicio es construir un query que compute la covarianza entre el retraso en el tiempo de salida (`ontime.DepDelay`) y el retraso en el tiempo de llegada (`ontime.ArrDelay`). Para eso trabaje los siguientes incisos:

Nota: recuerde que puede utilizar `%%sql` en Jupyter Notebook para escribir queries utilizando múltiples líneas.

##### 6.a

Escriba un query para computar el término $1/(n-1)$ de la tabla de vuelos, en donde $n$ representa la cantidad de vuelos disponibles:

_Respuesta:_ Su query debería retornar el valor `1.42658908114e-07`

##### 6.b

Escriba un query para que devuelva la siguiente el producto $(x_i - \overline{x})(y_i - \overline{y})$ para los primeros 10 registros de la tabla de vuelos:

#### 6.c

Escriba un query que utilice la consulta anterior compute la suma de los $n$ registros de la tabla de vuelos, es decir el término

\begin{equation}
    \sum_{i=1}^{n}(x_i - \overline{x})(y_i - \overline{y})
\end{equation}

##### 6.d

Escriba un query que utilizando las consultas de los incisos (a) y (c) para computar la covarianza.

_Respuesta: La covarianza entre el tiempo de salida y el tiempo de llegada en los vuelos del dataset es de 1,235.72_